In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2024-01-12T12:49:43.950535-05:00

Python implementation: CPython
Python version       : 3.7.11
IPython version      : 7.10.1

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 4.15.0-161-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 40
Architecture: 64bit



In [3]:
%watermark --gpu

GPU Info: 
  GPU 0: Tesla V100-DGXS-32GB
  GPU 1: Tesla V100-DGXS-32GB
  GPU 2: Tesla V100-DGXS-32GB
  GPU 3: Tesla V100-DGXS-32GB



In [4]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
from dask.delayed import delayed
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

import optuna
import gc
import logging

In [5]:
%watermark --iversions

logging: 0.5.1.2
pandas : 1.3.5
optuna : 2.10.1
dask   : 2022.1.0
xgboost: 1.6.2
numpy  : 1.21.6



In [6]:
!nvidia-smi -L

GPU 0: Tesla V100-DGXS-32GB (UUID: GPU-d752af09-1f62-bf3b-4f70-78b84e9e41f6)
GPU 1: Tesla V100-DGXS-32GB (UUID: GPU-054a4a35-f98a-3ebc-9100-0f697c246b43)
GPU 2: Tesla V100-DGXS-32GB (UUID: GPU-454525d4-bebd-7fb8-0ba3-3b85e2f99abd)
GPU 3: Tesla V100-DGXS-32GB (UUID: GPU-af0fb74e-f5eb-0833-17ff-494cc6bdcee1)


In [7]:
cluster = LocalCUDACluster(n_workers=4)
client = Client(cluster)

In [8]:
!nvidia-smi

Fri Jan 12 12:50:08 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.142.00   Driver Version: 450.142.00   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-DGXS...  On   | 00000000:07:00.0  On |                    0 |
| N/A   35C    P0    50W / 300W |   1639MiB / 32505MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-DGXS...  On   | 00000000:08:00.0 Off |                    0 |
| N/A   

In [9]:
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses
    
    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)
 
def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

In [10]:
train_folds = []
val_folds = []
train_ys = []
val_ys = []

for i in range(5):
    print(f'Loading fold {i}')
    train_fold_d = delayed(pd.read_csv)(f'../input/xgtrain_fold_{i}_drop_shap.csv.gz')
    train_fold = dd.from_delayed(train_fold_d)
    
    val_fold_d = delayed(pd.read_csv)(f'../input/xgval_fold_{i}_drop_shap.csv.gz')
    val_fold = dd.from_delayed(val_fold_d)
    
    
    train_y = train_fold['target']
    train_fold = train_fold[train_fold.columns.difference(['target'])]
    
    val_y = val_fold['target']
    val_fold = val_fold[val_fold.columns.difference(['target'])]
    
    train_folds.append(train_fold)
    val_folds.append(val_fold)
    
    train_ys.append(train_y)
    val_ys.append(val_y)

Loading fold 0
Loading fold 1
Loading fold 2
Loading fold 3
Loading fold 4


In [11]:
train = pd.read_csv('../input/train.csv.zip')

target = train['target'].values
target

array([0, 0, 0, ..., 0, 0, 0])

In [12]:
train_oof = np.zeros((target.shape[0],))

num_round = 1000

def objective(trial):
        
    params = {
        'objective': trial.suggest_categorical('objective',['binary:logistic']),
        'tree_method': trial.suggest_categorical('tree_method',['gpu_hist']),  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001,0.1),
        #'n_estimators': trial.suggest_categorical('n_estimators', [1000]),
        'max_depth': trial.suggest_int('max_depth', 3, 25),
        #'random_state': trial.suggest_categorical('random_state', [24,48,2020]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'eval_metric': trial.suggest_categorical('eval_metric',['logloss']),

    }

    kf = KFold(5, shuffle=True, random_state=137)

    for i, (train_index, val_index) in enumerate(kf.split(train,target)):
        dtrain = xgb.dask.DaskDMatrix(client, train_folds[i].values, train_ys[i], enable_categorical=True)
        dval = xgb.dask.DaskDMatrix(client, val_folds[i].values, val_ys[i], enable_categorical=True)
        
        output = xgb.dask.train(client, params, dtrain, num_round)
        booster = output['booster']  # booster is the trained model
        booster.set_param({'predictor': 'gpu_predictor'})
        predictions = xgb.dask.predict(client, booster, dval)
        predictions = predictions.compute()
        train_oof[val_index] = predictions
        del dtrain, dval, output
        gc.collect()
        gc.collect()

    gini = gini_normalized(target, train_oof)
    
    return gini

In [13]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)  # Setup the root logger.
logger.addHandler(logging.FileHandler("optuna_xgb_output_3.log", mode="w"))

optuna.logging.enable_propagation()  # Propagate logs to the root logger.
optuna.logging.disable_default_handler()  # Stop showing logs in sys.stderr.

study = optuna.create_study(storage="sqlite:///xgb_optuna_porto_3.db", study_name="five_fold_optuna_xgb_3", direction='maximize')

In [14]:
%%time
logger.info("Start optimization.")
study.optimize(objective, n_trials=3)

/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
distributed.utils_perf - WARNING - full garbage collections took 46% CPU time recently (threshold: 10%)


CPU times: user 1min, sys: 11.6 s, total: 1min 11s
Wall time: 11min 27s


In [15]:
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.head()

,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_objective,params_subsample,params_tree_method,state
0,0,0.251517,5.904816,0.598923,logloss,0.156677,0.002723,7,260,binary:logistic,0.464757,gpu_hist,COMPLETE
1,1,0.278858,0.688639,0.847375,logloss,0.036075,0.006552,22,95,binary:logistic,0.877548,gpu_hist,COMPLETE
2,2,0.279854,1.520039,0.932739,logloss,0.040527,0.007199,23,154,binary:logistic,0.778910,gpu_hist,COMPLETE


In [16]:
%%time
study.optimize(objective, n_trials=5)
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.to_csv('optuna_xgb_output_3.csv', index=False)
df.head(8)

/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
distributed.utils_perf - WARNING - full garbage collections took 70% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 71% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 71% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 71% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 71% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 72% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 72% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 72% CPU time recently (thre

CPU times: user 38.4 s, sys: 7.14 s, total: 45.5 s
Wall time: 5min 45s


,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_objective,params_subsample,params_tree_method,state
0,0,0.251517,5.904816,0.598923,logloss,0.156677,0.002723,7,260,binary:logistic,0.464757,gpu_hist,COMPLETE
1,1,0.278858,0.688639,0.847375,logloss,0.036075,0.006552,22,95,binary:logistic,0.877548,gpu_hist,COMPLETE
2,2,0.279854,1.520039,0.932739,logloss,0.040527,0.007199,23,154,binary:logistic,0.778910,gpu_hist,COMPLETE
3,3,0.267726,0.002150,0.499880,logloss,0.014537,0.003140,7,75,binary:logistic,0.722202,gpu_hist,COMPLETE
4,4,0.271960,0.083991,0.454886,logloss,0.703989,0.014852,18,61,binary:logistic,0.546907,gpu_hist,COMPLETE
5,5,0.268354,0.005115,0.343452,logloss,0.764771,0.003464,8,118,binary:logistic,0.838139,gpu_hist,COMPLETE
6,6,0.273905,0.004593,0.555147,logloss,0.029696,0.020644,18,186,binary:logistic,0.994668,gpu_hist,COMPLETE
7,7,0.277930,0.007494,0.502458,logloss,0.012670,0.015709,6,26,binary:logistic,0.442279,gpu_hist,COMPLETE


In [17]:
study.best_trial.params

{'alpha': 1.5200392958092102,
 'colsample_bytree': 0.932739072014442,
 'eval_metric': 'logloss',
 'lambda': 0.04052684192430588,
 'learning_rate': 0.00719883208099798,
 'max_depth': 23,
 'min_child_weight': 154,
 'objective': 'binary:logistic',
 'subsample': 0.7789097702122305,
 'tree_method': 'gpu_hist'}

In [18]:
df.value.max()

0.2798542997585565

In [19]:
%%time
study.optimize(objective, n_trials=100)
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.to_csv('optuna_xgb_output_3.csv', index=False)
df.head(20)

/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
distributed.utils_perf - WARNING - full garbage collections took 76% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 76% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 76% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 76% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 76% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 76% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 76% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 76% CPU time recently (thre

CPU times: user 18min 59s, sys: 3min 3s, total: 22min 2s
Wall time: 2h 44min 6s


,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_objective,params_subsample,params_tree_method,state
0,0,0.251517,5.904816,0.598923,logloss,0.156677,0.002723,7,260,binary:logistic,0.464757,gpu_hist,COMPLETE
1,1,0.278858,0.688639,0.847375,logloss,0.036075,0.006552,22,95,binary:logistic,0.877548,gpu_hist,COMPLETE
2,2,0.279854,1.520039,0.932739,logloss,0.040527,0.007199,23,154,binary:logistic,0.778910,gpu_hist,COMPLETE
3,3,0.267726,0.002150,0.499880,logloss,0.014537,0.003140,7,75,binary:logistic,0.722202,gpu_hist,COMPLETE
4,4,0.271960,0.083991,0.454886,logloss,0.703989,0.014852,18,61,binary:logistic,0.546907,gpu_hist,COMPLETE
5,5,0.268354,0.005115,0.343452,logloss,0.764771,0.003464,8,118,binary:logistic,0.838139,gpu_hist,COMPLETE
6,6,0.273905,0.004593,0.555147,logloss,0.029696,0.020644,18,186,binary:logistic,0.994668,gpu_hist,COMPLETE
7,7,0.277930,0.007494,0.502458,logloss,0.012670,0.015709,6,26,binary:logistic,0.442279,gpu_hist,COMPLETE
8,8,0.280316,0.021638,0.634273,logloss,0.003631,0.015195,5,32,binary:logistic,0.546882,gpu_hist,COMPLETE
9,9,0.264394,0.106545,0.884406,logloss,7.350213,0.033048,19,183,binary:logistic,0.752878,gpu_hist,COMPLETE


In [20]:
df.value.max()

0.28191195904253175

In [21]:
study.best_trial.params

{'alpha': 0.006772162862795028,
 'colsample_bytree': 0.609129374560866,
 'eval_metric': 'logloss',
 'lambda': 9.09234772552394,
 'learning_rate': 0.013588218801335329,
 'max_depth': 7,
 'min_child_weight': 110,
 'objective': 'binary:logistic',
 'subsample': 0.9656505461499723,
 'tree_method': 'gpu_hist'}

In [22]:
%%time
study.optimize(objective, n_trials=100)
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.to_csv('optuna_xgb_output_3.csv', index=False)
df.head(20)

/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
distributed.utils_perf - WARNING - full garbage collections took 88% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 88% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 88% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 88% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 88% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 88% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 88% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 88% CPU time recently (thre

CPU times: user 17min 53s, sys: 1min 39s, total: 19min 33s
Wall time: 1h 31min 38s


,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_objective,params_subsample,params_tree_method,state
0,0,0.251517,5.904816,0.598923,logloss,0.156677,0.002723,7,260,binary:logistic,0.464757,gpu_hist,COMPLETE
1,1,0.278858,0.688639,0.847375,logloss,0.036075,0.006552,22,95,binary:logistic,0.877548,gpu_hist,COMPLETE
2,2,0.279854,1.520039,0.932739,logloss,0.040527,0.007199,23,154,binary:logistic,0.778910,gpu_hist,COMPLETE
3,3,0.267726,0.002150,0.499880,logloss,0.014537,0.003140,7,75,binary:logistic,0.722202,gpu_hist,COMPLETE
4,4,0.271960,0.083991,0.454886,logloss,0.703989,0.014852,18,61,binary:logistic,0.546907,gpu_hist,COMPLETE
5,5,0.268354,0.005115,0.343452,logloss,0.764771,0.003464,8,118,binary:logistic,0.838139,gpu_hist,COMPLETE
6,6,0.273905,0.004593,0.555147,logloss,0.029696,0.020644,18,186,binary:logistic,0.994668,gpu_hist,COMPLETE
7,7,0.277930,0.007494,0.502458,logloss,0.012670,0.015709,6,26,binary:logistic,0.442279,gpu_hist,COMPLETE
8,8,0.280316,0.021638,0.634273,logloss,0.003631,0.015195,5,32,binary:logistic,0.546882,gpu_hist,COMPLETE
9,9,0.264394,0.106545,0.884406,logloss,7.350213,0.033048,19,183,binary:logistic,0.752878,gpu_hist,COMPLETE


In [23]:
study.best_trial.params

{'alpha': 0.23303245048392487,
 'colsample_bytree': 0.5908825877205581,
 'eval_metric': 'logloss',
 'lambda': 0.019491307234673853,
 'learning_rate': 0.010888507556362797,
 'max_depth': 8,
 'min_child_weight': 130,
 'objective': 'binary:logistic',
 'subsample': 0.9430117188563832,
 'tree_method': 'gpu_hist'}

In [24]:
df.value.max()

0.282082833433646